# Разбор домашнего задания Парсинг данных
# Pro

Возьмите базу квартир (moscow.csv), проведите парсинг и очистку данных:

● Уберите строки “Для заметок”

● Уберите столбцы “ГРМ” и “Примечания”

● Для столбца “Балкон” добавьте категорию “Отсутствует” (вместо NaN)

● Значения столбца “Бонус агенту” переведите в тысячи рублей,а пустые значения
заполните нулями

● В столбец “Площадь” запишите общую площадь всех помещений

● Для столбца “Санузел” заполните отсутствующие значения строкой “Нет данных”

● Столбец “Цена” переведите в тысячи рублей

In [ ]:
# Статический вывод графики (графики отображаются в той ячейке, в которой используется plt.show())
import pandas as pd # Импортируем библиотеку обработки и анализа данных pandas
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
import datetime                 # Импортируем модуль datetime для обработки данных в формете даты
import numpy as np              # Импортируем библиотеку numpy
from google.colab import files  # Импортируем библиотеку files для работы с файлами
import re                       # Регулярные выражения
import csv


# Рисовать графики сразу же
%matplotlib inline

In [ ]:
# Для загрузки данных используем drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving moscow.csv to moscow.csv


In [ ]:
mos = pd.read_csv('/content/drive/MyDrive/moscow.csv',  sep=";") 


In [ ]:
mos.head()

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",ГРМ,Бонус агенту,Дата,Кол-во дней в экспозиции,Источник,Примечание
0,1,Шелепиха м.,12п,35/37 М,NaN,2,64.1/23/20,19500000.0,NaN,NaN,25.07.2019,1.0,Привлекательные,Собственность оформлена! Ключи на руках! Опера...
1,Для заметок:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,NaN,NaN,25.07.2019,2.0,Привлекательные,Зеленоград к.1820 однокомнатная квартира общ.3...
3,Для заметок:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,NaN,NaN,25.07.2019,4.0,Привлекательные,"Уютная, светлая квартира, застекленный балкон ..."


In [ ]:
mos1=mos.copy() # Копию создавал для второго способа преобразования данных по площади. Отражен в конце работы.
mos1

In [ ]:
mos = mos.drop(mos.columns[[8,13]],axis=1 ) # удаляем неинтересующие нас столбцы: ГРМ и Примечание


In [ ]:
mos.Балкон = mos.Балкон.fillna('Отсутствует')               # Для столбца “Балкон” добавьте категорию “Отсутствует” (вместо NaN)
mos["Бонус агенту"] = mos["Бонус агенту"].fillna('0')       # Значения столбца “Бонус агенту” пустые значения заполните нулями
mos.Санузел = mos.Санузел.fillna('Нет данных')              # Для столбца “Санузел” заполните отсутствующие значения строкой “Нет данных”


In [ ]:
mos = mos.rename(columns={"Цена, руб.": "Цена, тыс. руб."})   # Столбец “Цена” переводим в тысячи рублей
mos['Цена, тыс. руб.'] = round(mos['Цена, тыс. руб.']/1000)



In [ ]:
# Agent bonus 
def getBonus(arg):
    #global salaryStr, num #т.к. используем, меняем в функции
    # Сначала получаем чистое число, убираем лишние знаки
    salaryStr = 0
    arg=str(arg)  
    salaryStr = int(re.sub(r'[^0-9]+', '', arg))
    curr_ = re.sub('[0-9]','',arg).replace('.','')

    # Конвертируем в рубли, если валюта
    if 'руб' in curr_ :   salaryStr = float(salaryStr)/1000, 'тыс руб.'
    if '%' in curr_ :   salaryStr = float(salaryStr), '%'
    
    num =salaryStr 
    return salaryStr

In [ ]:
getBonus('25 000 руб.')

(25.0, 'тыс руб.')

In [ ]:
Bonus = getBonus

In [ ]:
mos['Бонус агенту'] = mos['Бонус агенту'].apply(Bonus)    #Перевел оплату агента из руб в тыс руб.

In [ ]:
mos.groupby('Площадь')['Площадь'].count()  #.iloc[400:425]


Площадь
10.06.2002    1
10.07.2002    1
10.08.2002    1
10.2/8/?      1
10.5/8.5/2    1
             ..
?/9.6/?       1
?/9/9         1
?/9/?         5
?/94/?        1
?/97/?        2
Name: Площадь, Length: 39702, dtype: int64

In [ ]:
mos = mos.drop(mos[mos['Комнат'].str.contains('\d\d.\w\w\w', na=False)].index, axis = 0) # Удаляю выбросы - даты вместо количества комнат

In [ ]:
mos = mos.drop(mos[mos['Площадь'].str.contains('\d\d.\d\d.\d\d\d\d', na=False)].index, axis = 0)   # Удаляю выбросы - даты вместо площади


In [ ]:
mos = mos.drop(mos[mos['Комнат'].str.contains('Для заметок', na=False)].index, axis = 0) # Убираю строки “Для заметок”

In [ ]:
mos

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, тыс. руб.",Бонус агенту,Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64.1/23/20,19500.0,0,25.07.2019,1.0,Привлекательные
2,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800.0,0,25.07.2019,2.0,Привлекательные
4,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990.0,0,25.07.2019,4.0,Привлекательные
6,1,Шаболовская м.,12п,8/12 ?,Б,Р,38/15/7,11200.0,0,25.07.2019,7.0,Привлекательные
8,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34/17/8,6250.0,0,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
252526,Ст,Некрасовка м.,4п,2/17 П,Отсутствует,Нет данных,19.8/?/?,2850.0,0,12.07.2019,14.0,AVITO.ru
252528,Ст,Авиамоторная м.,6п,2/8 К,Отсутствует,С,20.4/18/?,3840.0,0,23.07.2019,14.0,Яндекс
252530,Ст,Коломенская м.,15п,1/9 П,Отсутствует,Нет данных,15.4/13.2/?,2900.0,0,12.07.2019,14.0,AVITO.ru
252532,Ст,Алтуфьево м.,5т,4/9 М,Отсутствует,Нет данных,26/10.9/7,5200.0,0,22.07.2019,14.0,AVITO.ru


In [ ]:
mos['Площадь'] = mos['Площадь'].str.extract(r'(\w+)', expand=False) #Вывел общую площадь в столбце "Площадь"

In [ ]:
mos

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, тыс. руб.",Бонус агенту,Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64,19500.0,0,25.07.2019,1.0,Привлекательные
2,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38,4800.0,0,25.07.2019,2.0,Привлекательные
4,1,Планерная м.,5т,11/22 П,Б,С,38,7990.0,0,25.07.2019,4.0,Привлекательные
6,1,Шаболовская м.,12п,8/12 ?,Б,Р,38,11200.0,0,25.07.2019,7.0,Привлекательные
8,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34,6250.0,0,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
252526,Ст,Некрасовка м.,4п,2/17 П,Отсутствует,Нет данных,19,2850.0,0,12.07.2019,14.0,AVITO.ru
252528,Ст,Авиамоторная м.,6п,2/8 К,Отсутствует,С,20,3840.0,0,23.07.2019,14.0,Яндекс
252530,Ст,Коломенская м.,15п,1/9 П,Отсутствует,Нет данных,15,2900.0,0,12.07.2019,14.0,AVITO.ru
252532,Ст,Алтуфьево м.,5т,4/9 М,Отсутствует,Нет данных,26,5200.0,0,22.07.2019,14.0,AVITO.ru


Второй способ преобразования в столбец с общей площадью. На мой взгляд он лучше подходит для последующей обработки т.к. содержит данные не str а float.

In [ ]:
mos1=mos.copy() # Копию создавал до преобразования данных по площади.
mos1

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, тыс. руб.",Бонус агенту,Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64.1/23/20,19500.0,0,25.07.2019,1.0,Привлекательные
2,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800.0,0,25.07.2019,2.0,Привлекательные
4,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990.0,0,25.07.2019,4.0,Привлекательные
6,1,Шаболовская м.,12п,8/12 ?,Б,Р,38/15/7,11200.0,0,25.07.2019,7.0,Привлекательные
8,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34/17/8,6250.0,0,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
252526,Ст,Некрасовка м.,4п,2/17 П,Отсутствует,Нет данных,19.8/?/?,2850.0,0,12.07.2019,14.0,AVITO.ru
252528,Ст,Авиамоторная м.,6п,2/8 К,Отсутствует,С,20.4/18/?,3840.0,0,23.07.2019,14.0,Яндекс
252530,Ст,Коломенская м.,15п,1/9 П,Отсутствует,Нет данных,15.4/13.2/?,2900.0,0,12.07.2019,14.0,AVITO.ru
252532,Ст,Алтуфьево м.,5т,4/9 М,Отсутствует,Нет данных,26/10.9/7,5200.0,0,22.07.2019,14.0,AVITO.ru


In [ ]:
mos1

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, тыс. руб.",Бонус агенту,Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64,19500.0,0,25.07.2019,1.0,Привлекательные
2,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38,4800.0,0,25.07.2019,2.0,Привлекательные
4,1,Планерная м.,5т,11/22 П,Б,С,38,7990.0,0,25.07.2019,4.0,Привлекательные
6,1,Шаболовская м.,12п,8/12 ?,Б,Р,38,11200.0,0,25.07.2019,7.0,Привлекательные
8,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34,6250.0,0,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
252526,Ст,Некрасовка м.,4п,2/17 П,Отсутствует,Нет данных,19,2850.0,0,12.07.2019,14.0,AVITO.ru
252528,Ст,Авиамоторная м.,6п,2/8 К,Отсутствует,С,20,3840.0,0,23.07.2019,14.0,Яндекс
252530,Ст,Коломенская м.,15п,1/9 П,Отсутствует,Нет данных,15,2900.0,0,12.07.2019,14.0,AVITO.ru
252532,Ст,Алтуфьево м.,5т,4/9 М,Отсутствует,Нет данных,26,5200.0,0,22.07.2019,14.0,AVITO.ru


In [ ]:
def getsquare(i):         # Преобразовал код в функцию
  i=re.split(r'/', i)
  arg=i[0]
  if arg != '?':
    arg=np.float_(arg)
    i = arg
    return i

In [ ]:
mos1['Площадь'] = mos1['Площадь'].apply(getsquare)

In [ ]:
mos1

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, тыс. руб.",Бонус агенту,Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64.1,19500.0,0,25.07.2019,1.0,Привлекательные
2,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4,4800.0,0,25.07.2019,2.0,Привлекательные
4,1,Планерная м.,5т,11/22 П,Б,С,38.2,7990.0,0,25.07.2019,4.0,Привлекательные
6,1,Шаболовская м.,12п,8/12 ?,Б,Р,38.0,11200.0,0,25.07.2019,7.0,Привлекательные
8,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34.0,6250.0,0,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
252526,Ст,Некрасовка м.,4п,2/17 П,Отсутствует,Нет данных,19.8,2850.0,0,12.07.2019,14.0,AVITO.ru
252528,Ст,Авиамоторная м.,6п,2/8 К,Отсутствует,С,20.4,3840.0,0,23.07.2019,14.0,Яндекс
252530,Ст,Коломенская м.,15п,1/9 П,Отсутствует,Нет данных,15.4,2900.0,0,12.07.2019,14.0,AVITO.ru
252532,Ст,Алтуфьево м.,5т,4/9 М,Отсутствует,Нет данных,26.0,5200.0,0,22.07.2019,14.0,AVITO.ru


In [ ]:
mos1['Площадь'].unique()

array([ 64.1,  38.4,  38.2, ..., 199.2,  11.9, 180.8])

Имеются данные "?" вида. Для дальнейшего использования их лучше тоже дропнуть. Но в задании этого не было.